In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

c:\Users\Ege\Documents\egemenkopuz\projects\inpainting-in-medical-imaging


In [3]:
%pwd

'c:\\Users\\Ege\\Documents\\egemenkopuz\\projects\\inpainting-in-medical-imaging'

In [4]:
from src.dataset import BraTSDataset
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
import torch
import yaml
import os
from glob import glob

# from sewar.full_ref import vifp, ssim
from sklearn.metrics import mean_squared_error
from piq import vif_p, ssim, psnr, gmsd, FID, multi_scale_ssim

from src.models.baseline import BaselineInpaintingGenerator

In [6]:
def map_grade(x: torch.Tensor, requires_grad: bool = False):
    out = torch.zeros(x.shape, dtype=torch.float, requires_grad=requires_grad).to(
        x.device
    )
    out[(x == 1).nonzero(as_tuple=True)] = 0.5
    out[(x == 2).nonzero(as_tuple=True)] = 0.75
    out[(x == 3).nonzero(as_tuple=True)] = 1.0
    return out

In [15]:
# checkpoints = {
#     "baseline25_epoch100": glob(
#         "final_results/baseline_inpainting 21-01-2022 16-44-21/*/checkpoints/*/network_inpainting_gen.pth.tar"
#     )
#     + glob(
#         "final_results/baseline_inpainting cont 10-02-2022 19-58-00/*/checkpoints/*/network_inpainting_gen.pth.tar"
#     ),
#     "baseline100_epoch50": glob(
#         "final_results/baseline_inpainting 100 13-02-2022 18-00-59/*/checkpoints/*/network_inpainting_gen.pth.tar"
#     ),
#     "fft25_epoch100": glob(
#         "final_results/baseline_inpainting fft 11-02-2022 08-30-50/*/checkpoints/*/network_inpainting_gen.pth.tar"
#     ),
#     "fft100_epoch50": glob(
#         "final_results/baseline_inpainting fft 100 13-02-2022 17-48-39/*/checkpoints/*/network_inpainting_gen.pth.tar"
#     ),
# }

checkpoints = {
    "baseline25_epoch100": [
        "final_results/baseline_inpainting 21-01-2022 16-44-21/21-01-2022 16-44-21/checkpoints/50/network_inpainting_gen.pth.tar",
        "final_results/baseline_inpainting 21-01-2022 16-44-21/21-01-2022 16-44-21/checkpoints/100/network_inpainting_gen.pth.tar",
        "final_results/baseline_inpainting cont 10-02-2022 19-58-00/10-02-2022 19-58-00/checkpoints/150/network_inpainting_gen.pth.tar",
        "final_results/baseline_inpainting cont 10-02-2022 19-58-00/10-02-2022 19-58-00/checkpoints/200/network_inpainting_gen.pth.tar",
    ],
    "baseline100_epoch50": [
        "final_results/baseline_inpainting 100 13-02-2022 18-00-59/13-02-2022 18-00-59/checkpoints/50/network_inpainting_gen.pth.tar",
        "final_results/baseline_inpainting 100 13-02-2022 18-00-59/13-02-2022 18-00-59/checkpoints/25/network_inpainting_gen.pth.tar",
    ],
    "fft25_epoch100": [
        "final_results/baseline_inpainting fft 11-02-2022 08-30-50/11-02-2022 08-30-50/checkpoints/50/network_inpainting_gen.pth.tar",
        "final_results/baseline_inpainting fft 11-02-2022 08-30-50/11-02-2022 08-30-50/checkpoints/100/network_inpainting_gen.pth.tar",
    ],
    "fft100_epoch50": [
        "final_results/baseline_inpainting fft 100 13-02-2022 17-48-39/13-02-2022 17-48-39/checkpoints/25/network_inpainting_gen.pth.tar",
        "final_results/baseline_inpainting fft 100 13-02-2022 17-48-39/13-02-2022 17-48-39/checkpoints/50/network_inpainting_gen.pth.tar",
    ],
}


In [16]:
device = torch.device(f"cuda:{torch.cuda.current_device()}") if torch.cuda.is_available() else "cpu"
# device = "cpu"

input_path = "resources/data/brats20/validation"
fixed_indices = "resources/indices/brats20_25p.yml"
output_path = "resources/inpainting_eval/"

with open(fixed_indices) as file:
    indices = yaml.safe_load(file)["val"]

In [17]:
df = pd.DataFrame(
    {"experiment": pd.Series(dtype="str"), "epoch": pd.Series(dtype="int")}
)

In [35]:
for exp_name, cps in checkpoints.items():
    for cp in cps:
        epoch_i = int(os.path.normpath(cp).split(os.sep)[-2])
        output_p = os.path.join(output_path, exp_name, str(epoch_i))
        os.makedirs(os.path.join(output_p, "t1"), exist_ok=True)
        os.makedirs(os.path.join(output_p, "t1ce"), exist_ok=True)
        os.makedirs(os.path.join(output_p, "t2"), exist_ok=True)
        os.makedirs(os.path.join(output_p, "flair"), exist_ok=True)

        inp_gen_cp = torch.load(cp)
        inp_gen = BaselineInpaintingGenerator().to(device)
        inp_gen.load_state_dict(inp_gen_cp["state_dict"])

        ssim_t1, ssim_t1ce, ssim_t2, ssim_flair = 0.0, 0.0, 0.0, 0.0
        ms_ssim_t1, ms_ssim_t1ce, ms_ssim_t2, ms_ssim_flair = 0.0, 0.0, 0.0, 0.0
        vifp_t1, vifp_t1ce, vifp_t2, vifp_flair = 0.0, 0.0, 0.0, 0.0
        psnr_t1, psnr_t1ce, psnr_t2, psnr_flair = 0.0, 0.0, 0.0, 0.0
        gmsd_t1, gmsd_t1ce, gmsd_t2, gmsd_flair = 0.0, 0.0, 0.0, 0.0
        rmse_t1, rmse_t1ce, rmse_t2, rmse_flair = 0.0, 0.0, 0.0, 0.0

        dataset = BraTSDataset(root=input_path, mode="val", fixed_indices=indices)
        loader = DataLoader(
            dataset=dataset, batch_size=1, shuffle=False, num_workers=0, pin_memory=True
        )

        pb = tqdm(loader, desc=f"{exp_name} - epoch: {epoch_i}")
        for i, batch in enumerate(pb):
            t_t1 = batch["t1"].to(device)
            t_t1ce = batch["t1ce"].to(device)
            t_t2 = batch["t2"].to(device)
            t_flair = batch["flair"].to(device)
            t_m_brain = batch["mask_brain_shape"].to(device)
            t_m_shape = batch["mask_tumor_shape"].to(device)
            t_m_grade_mapped = map_grade(batch["mask_tumor_grade"]).to(device)
            t_m_background = 1 - t_m_shape
            t_brain = torch.cat((t_t1, t_t1ce, t_t2, t_flair), 1)
            brain_w_blank_mask = t_brain * t_m_background

            p_brain = inp_gen(brain_w_blank_mask, t_m_grade_mapped)
            out = p_brain * t_m_shape + t_brain * t_m_background
            p_t1, p_t1ce, p_t2, p_flair = torch.split(
                out, split_size_or_sections=1, dim=1
            )

            save_image(p_t1[0][0], fp=f"{output_p}/t1/{i}.png")
            save_image(p_t1ce[0][0], fp=f"{output_p}/t1ce/{i}.png")
            save_image(p_t2[0][0], fp=f"{output_p}/t2/{i}.png")
            save_image(p_flair[0][0], fp=f"{output_p}/flair/{i}.png")

            t_t1 = t_t1.cpu().detach()
            t_t1ce = t_t1ce.cpu().detach()  
            t_t2 = t_t2.cpu().detach()
            t_flair = t_flair.cpu().detach()

            p_t1 = p_t1.cpu().detach()
            p_t1ce = p_t1ce.cpu().detach()
            p_t2 = p_t2.cpu().detach()
            p_flair = p_flair.cpu().detach()

            ssim_t1 += ssim(t_t1, p_t1, data_range=1.0)
            ssim_t1ce += ssim(t_t1ce, p_t1ce, data_range=1.0)
            ssim_t2 += ssim(t_t2, p_t2, data_range=1.0)
            ssim_flair += ssim(t_flair, p_flair, data_range=1.0)

            ms_ssim_t1 += multi_scale_ssim(t_t1, p_t1, data_range=1.0)
            ms_ssim_t1ce += multi_scale_ssim(t_t1ce, p_t1ce, data_range=1.0)
            ms_ssim_t2 += multi_scale_ssim(t_t2, p_t2, data_range=1.0)
            ms_ssim_flair += multi_scale_ssim(t_flair, p_flair, data_range=1.0)

            vifp_t1 += vif_p(t_t1, p_t1, data_range=1.0)
            vifp_t1ce += vif_p(t_t1ce, p_t1ce, data_range=1.0)
            vifp_t2 += vif_p(t_t2, p_t2, data_range=1.0)
            vifp_flair += vif_p(t_flair, p_flair, data_range=1.0)

            psnr_t1 += psnr(t_t1, p_t1, data_range=1.0)
            psnr_t1ce += psnr(t_t1ce, p_t1ce, data_range=1.0)
            psnr_t2 += psnr(t_t2, p_t2, data_range=1.0)
            psnr_flair += psnr(t_flair, p_flair, data_range=1.0)

            gmsd_t1 += gmsd(t_t1, p_t1, data_range=1.0)
            gmsd_t1ce += gmsd(t_t1ce, p_t1ce, data_range=1.0)
            gmsd_t2 += gmsd(t_t2, p_t2, data_range=1.0)
            gmsd_flair += gmsd(t_flair, p_flair, data_range=1.0)

            rmse_t1 += mean_squared_error(
                t_t1.numpy()[0][0], p_t1.numpy()[0][0], squared=False
            )
            rmse_t1ce += mean_squared_error(
                t_t1ce.numpy()[0][0], p_t1ce.numpy()[0][0], squared=False
            )
            rmse_t2 += mean_squared_error(
                t_t2.numpy()[0][0], p_t2.numpy()[0][0], squared=False
            )
            rmse_flair += mean_squared_error(
                t_flair.numpy()[0][0], p_flair.numpy()[0][0], squared=False
            )

        total_b = len(pb)

        ssim_t1 /= total_b
        ssim_t1ce /= total_b
        ssim_t2 /= total_b
        ssim_flair /= total_b
        ssim_avg = np.mean([ssim_t1, ssim_t1ce, ssim_t2, ssim_flair], dtype=np.float64)

        ms_ssim_t1 /= total_b
        ms_ssim_t1ce /= total_b
        ms_ssim_t2 /= total_b
        ms_ssim_flair /= total_b
        ms_ssim_avg = np.mean([ms_ssim_t1, ms_ssim_t1ce, ms_ssim_t2, ms_ssim_flair], dtype=np.float64)

        vifp_t1 /= total_b
        vifp_t1ce /= total_b
        vifp_t2 /= total_b
        vifp_flair /= total_b
        vifp_avg = np.mean([vifp_t1, vifp_t1ce, vifp_t2, vifp_flair], dtype=np.float64)

        psnr_t1 /= total_b
        psnr_t1ce /= total_b
        psnr_t2 /= total_b
        psnr_flair /= total_b
        psnr_avg = np.mean([psnr_t1, psnr_t1ce, psnr_t2, psnr_flair], dtype=np.float64)

        gmsd_t1 /= total_b
        gmsd_t1ce /= total_b
        gmsd_t2 /= total_b
        gmsd_flair /= total_b
        gmsd_avg = np.mean([gmsd_t1, gmsd_t1ce, gmsd_t2, gmsd_flair], dtype=np.float64)

        rmse_t1 /= total_b
        rmse_t1ce /= total_b
        rmse_t2 /= total_b
        rmse_flair /= total_b
        rmse_avg = np.mean([rmse_t1, rmse_t1ce, rmse_t2, rmse_flair], dtype=np.float64)

        df = df.append(
            {
                "experiment": exp_name,
                "epoch": epoch_i,
                "ssim_avg": ssim_avg,
                "ssim_t1": ssim_t1.item(),
                "ssim_t1ce": ssim_t1ce.item(),
                "ssim_t2": ssim_t2.item(),
                "ssim_flair": ssim_flair.item(),
                "ms_ssim_avg": ms_ssim_avg,
                "ms_ssim_t1": ms_ssim_t1.item(),
                "ms_ssim_t1ce": ms_ssim_t1ce.item(),
                "ms_ssim_t2": ms_ssim_t2.item(),
                "ms_ssim_flair": ms_ssim_flair.item(),
                "vifp_avg": vifp_avg,
                "vifp_t1": vifp_t1.item(),
                "vifp_t1ce": vifp_t1ce.item(),
                "vifp_t2": vifp_t2.item(),
                "vifp_flair": vifp_flair.item(),
                "psnr_avg": psnr_avg,
                "psnr_t1": psnr_t1.item(),
                "psnr_t1ce": psnr_t1ce.item(),
                "psnr_t2": psnr_t2.item(),
                "psnr_flair": psnr_flair.item(),
                "gmsd_avg": gmsd_avg,
                "gmsd_t1": gmsd_t1.item(),
                "gmsd_t1ce": gmsd_t1ce.item(),
                "gmsd_t2": gmsd_t2.item(),
                "gmsd_flair": gmsd_flair.item(),
                "rmse_avg": rmse_avg,
                "rmse_t1": rmse_t1,
                "rmse_t1ce": rmse_t1ce,
                "rmse_t2": rmse_t2,
                "rmse_flair": rmse_flair,
            },
            ignore_index=True,
        )

df.to_csv("inpainting_scalars_only_last.csv")


fft100_epoch50 - epoch: 50: 100%|██████████| 531/531 [04:33<00:00,  1.94it/s]


In [36]:
df

,experiment,epoch,gmsd_avg,gmsd_flair,gmsd_t1,gmsd_t1ce,gmsd_t2,ms_ssim_avg,ms_ssim_flair,ms_ssim_t1,...,ssim_avg,ssim_flair,ssim_t1,ssim_t1ce,ssim_t2,vifp_avg,vifp_flair,vifp_t1,vifp_t1ce,vifp_t2
0,baseline25_epoch100,50,0.039045,0.045209,0.035634,0.031794,0.043545,0.984636,0.982181,0.985043,...,0.984636,0.982181,0.985043,0.987684,0.983634,0.892985,0.874966,0.906470,0.901971,0.888532
1,baseline25_epoch100,100,0.038798,0.044341,0.035628,0.031383,0.043841,0.983834,0.981362,0.984315,...,0.983834,0.981362,0.984315,0.986930,0.982728,0.886562,0.868165,0.901401,0.893908,0.882773
2,baseline25_epoch100,150,0.038670,0.044114,0.035334,0.031733,0.043499,0.983474,0.981146,0.983845,...,0.983474,0.981146,0.983845,0.986559,0.982347,0.885253,0.866809,0.900027,0.893563,0.880614
3,baseline25_epoch100,200,0.038883,0.044430,0.035643,0.031752,0.043708,0.983171,0.980809,0.983580,...,0.983171,0.980809,0.983580,0.986211,0.982083,0.883119,0.864206,0.898424,0.890253,0.879594
4,baseline100_epoch50,50,0.039022,0.045443,0.035236,0.031945,0.043464,0.984670,0.982004,0.985518,...,0.984670,0.982004,0.985518,0.987557,0.983599,0.893370,0.873902,0.909271,0.902804,0.887503
5,baseline100_epoch50,25,0.039256,0.046353,0.035350,0.031544,0.043776,0.985182,0.982490,0.985840,...,0.985182,0.982490,0.985840,0.988160,0.984239,0.897142,0.876850,0.911511,0.907282,0.892927
6,fft25_epoch100,50,0.039542,0.046953,0.035645,0.031511,0.044058,0.984513,0.982028,0.985050,...,0.984513,0.982028,0.985050,0.987526,0.983449,0.892717,0.876948,0.906848,0.900060,0.887012
7,fft25_epoch100,100,0.039174,0.045595,0.035635,0.031385,0.044080,0.983790,0.981412,0.984110,...,0.983790,0.981412,0.984110,0.986978,0.982661,0.886508,0.868185,0.901471,0.895412,0.880963
8,fft100_epoch50,25,0.039216,0.046702,0.035041,0.032154,0.042969,0.985137,0.982468,0.985982,...,0.985137,0.982468,0.985982,0.988017,0.984079,0.896397,0.876454,0.911863,0.906879,0.890394
9,fft100_epoch50,50,0.039039,0.046015,0.034766,0.031710,0.043665,0.984564,0.981932,0.985229,...,0.984564,0.981932,0.985229,0.987483,0.983613,0.893264,0.873764,0.906384,0.902704,0.890204


In [ ]:
df_fid = pd.DataFrame(
    {"experiment": pd.Series(dtype="str"), "epoch": pd.Series(dtype="int")}
)

for exp_name, cps in checkpoints.items():
    for cp in cps:
        epoch_i = int(os.path.normpath(cp).split(os.sep)[-2])
        output_p = os.path.join(output_path, exp_name, str(epoch_i))

        fid_t1, fid_t1ce, fid_t2, fid_flair = 0.0, 0.0, 0.0, 0.0

        for t in ["t1", "t1ce", "t2", "flair"]:
            org_dataloader = None
            pred_dataloader = None

            fid_metric = FID()
            first_feats = fid_metric.compute_feats(org_dataloader)
            second_feats = fid_metric.compute_feats(pred_dataloader)
            fid: torch.Tensor = fid_metric(first_feats, second_feats)

            if t == "t1": 
                fid_t1 = fid.item()
            elif t == "t1ce": 
                fid_t1ce = fid.item()
            elif t == "t2": 
                fid_t2 = fid.item()
            else:
                fid_flair = fid.item()

        fid_avg = (fid_t1 + fid_t1ce + fid_t2 + fid_flair) / 4

        df_fid = df_fid.append(
            {
                "experiment": exp_name,
                "epoch": epoch_i,
                "fid_avg": fid_avg,
                "fid_t1": fid_t1,
                "fid_t1ce": fid_t1ce,
                "fid_t2": fid_t2,
                "fid_flair":fid_flair
            },
            ignore_index=True,
        )

df_fid.to_csv("inpainting_scalars_only_last_fid.csv")


# Transfer original data

In [5]:
org_data_dump_path_t1= "resources/fid/original_val/t1"
org_data_dump_path_t1ce= "resources/fid/original_val/t1ce"
org_data_dump_path_t2= "resources/fid/original_val/t2"
org_data_dump_path_flair= "resources/fid/original_val/flair"

os.makedirs(org_data_dump_path_t1,exist_ok=True)
os.makedirs(org_data_dump_path_t1ce,exist_ok=True)
os.makedirs(org_data_dump_path_t2,exist_ok=True)
os.makedirs(org_data_dump_path_flair,exist_ok=True)

dataset = BraTSDataset(root=input_path, mode="val", fixed_indices=indices)
loader = DataLoader(dataset=dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

pb = tqdm(loader)
sample_count = 0
for batch in pb:
    t_t1 = batch["t1"]
    t_t1ce = batch["t1ce"]
    t_t2 = batch["t2"]
    t_flair = batch["flair"]

    for i in range(t_t1.shape[0]):
        save_image(t_t1[i][0], fp=f"{org_data_dump_path_t1}/{sample_count}.png")
        save_image(t_t1ce[i][0], fp=f"{org_data_dump_path_t1ce}/{sample_count}.png")
        save_image(t_t2[i][0], fp=f"{org_data_dump_path_t2}/{sample_count}.png")
        save_image(t_flair[i][0], fp=f"{org_data_dump_path_flair}/{sample_count}.png")
        sample_count += 1

100%|██████████| 74/74 [05:04<00:00,  4.12s/it]


# Create a dataset from checkpoint

In [48]:
gen_cp_path = "results/baseline/21-01-2022 16-44-21/checkpoints/100/network_inpainting_gen.pth.tar"

inp_gen_cp = torch.load(gen_cp_path)
inp_gen = BaselineInpaintingGenerator().to(device)
inp_gen.load_state_dict(inp_gen_cp["state_dict"])
# inp_gen.eval()

<All keys matched successfully>

In [49]:
output_path = "resources/fid/baseline_val"

output_path_t1 = os.path.join(output_path,"t1")
output_path_t1ce = os.path.join(output_path,"t1ce")
output_path_t2 = os.path.join(output_path,"t2")
output_path_flair = os.path.join(output_path,"flair")

os.makedirs(output_path_t1,exist_ok=True)
os.makedirs(output_path_t1ce,exist_ok=True)
os.makedirs(output_path_t2,exist_ok=True)
os.makedirs(output_path_flair,exist_ok=True)

dataset = BraTSDataset(root=input_path, mode="test", fixed_indices=indices)
loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False)

In [50]:
pb = tqdm(loader)
sample_count = 0
for batch in pb:
    t_t1 = batch["t1"].to(device)
    t_t1ce = batch["t1ce"].to(device)
    t_t2 = batch["t2"].to(device)
    t_flair = batch["flair"].to(device)
    t_m_brain = batch["mask_brain_shape"].to(device)
    t_m_shape = batch["mask_tumor_shape"].to(device)
    t_m_grade_mapped = map_grade(batch["mask_tumor_grade"]).to(device)
    t_m_background = 1 - t_m_shape
    t_brain = torch.cat((t_t1, t_t1ce, t_t2, t_flair), 1)
    brain_w_blank_mask = t_brain * t_m_background

    p_brain = inp_gen(brain_w_blank_mask, t_m_grade_mapped)
    out = p_brain * t_m_shape + t_brain * t_m_background
    t1, t1ce, t2, flair = torch.split(out, split_size_or_sections=1, dim=1)
    
    for i in range(t_t1.shape[0]):
        save_image(t1[i], fp=f"{output_path_t1}/{sample_count}.png")
        save_image(t1ce[i], fp=f"{output_path_t1ce}/{sample_count}.png")
        save_image(t2[i], fp=f"{output_path_t2}/{sample_count}.png")
        save_image(flair[i], fp=f"{output_path_flair}/{sample_count}.png")
        sample_count += 1

100%|██████████| 4686/4686 [11:13<00:00,  6.96it/s]


# FID calculations

## baseline25

In [23]:
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/50/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/50/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/50/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/50/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:21<00:00, 24.69it/s]


FID:  3.739898363986697


100%|██████████| 531/531 [00:21<00:00, 24.52it/s]


FID:  3.61967893264935


100%|██████████| 531/531 [00:20<00:00, 26.51it/s]


FID:  2.4271964313463457


100%|██████████| 531/531 [00:21<00:00, 25.09it/s]


FID:  4.9315633435397785


In [34]:
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/95/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/95/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/95/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/95/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:25<00:00, 20.45it/s]


FID:  1.3039606534730126


100%|██████████| 531/531 [00:25<00:00, 21.14it/s]


FID:  1.5510190791481477


100%|██████████| 531/531 [00:20<00:00, 26.48it/s]


FID:  0.9357614616201886


100%|██████████| 531/531 [00:20<00:00, 26.42it/s]


FID:  1.7225809989778895


In [25]:
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/150/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/150/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/150/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/150/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:21<00:00, 24.46it/s]


FID:  2.1265700919149424


100%|██████████| 531/531 [00:20<00:00, 25.65it/s]


FID:  2.0094137306051323


100%|██████████| 531/531 [00:20<00:00, 26.54it/s]


FID:  1.223871717844247


100%|██████████| 531/531 [00:21<00:00, 25.16it/s]


FID:  2.510254229232231


In [26]:
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/200/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/200/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/200/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline25_epoch100/200/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:20<00:00, 25.78it/s]


FID:  2.6362881427830303


100%|██████████| 531/531 [00:20<00:00, 25.80it/s]


FID:  2.7715125629683826


100%|██████████| 531/531 [00:21<00:00, 24.64it/s]


FID:  1.522992247770759


100%|██████████| 531/531 [00:22<00:00, 23.75it/s]


FID:  3.262191305283835


## baseline100

In [27]:
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/25/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/25/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/25/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/25/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:20<00:00, 25.53it/s]


FID:  7.1517692044566274


100%|██████████| 531/531 [00:20<00:00, 26.43it/s]


FID:  7.570413118819886


100%|██████████| 531/531 [00:20<00:00, 26.17it/s]


FID:  4.884765980235471


100%|██████████| 531/531 [00:19<00:00, 26.63it/s]


FID:  9.272935945356835


In [28]:
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/50/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/50/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/50/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/baseline100_epoch50/50/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:20<00:00, 26.25it/s]


FID:  5.944595280816685


100%|██████████| 531/531 [00:19<00:00, 26.66it/s]


FID:  6.435841006876018


100%|██████████| 531/531 [00:20<00:00, 26.15it/s]


FID:  4.002480941558019


100%|██████████| 531/531 [00:20<00:00, 26.47it/s]


FID:  8.454164204194086


## fft25

In [29]:
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/50/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/50/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/50/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/50/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:19<00:00, 26.62it/s]


FID:  5.836685245464565


100%|██████████| 531/531 [00:22<00:00, 23.49it/s]


FID:  5.81745288756602


100%|██████████| 531/531 [00:23<00:00, 22.84it/s]


FID:  4.028647513176935


100%|██████████| 531/531 [00:23<00:00, 22.42it/s]


FID:  8.304132165911824


In [37]:
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/95/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/95/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/95/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft25_epoch100/95/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:28<00:00, 18.84it/s]


FID:  4.684026823722121


100%|██████████| 531/531 [00:26<00:00, 19.94it/s]


FID:  4.887460010344967


100%|██████████| 531/531 [00:26<00:00, 20.03it/s]


FID:  3.387399876090228


100%|██████████| 531/531 [00:27<00:00, 19.25it/s]


FID:  7.195166964237217


## fft100

In [31]:
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/25/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/25/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/25/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/25/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:20<00:00, 25.78it/s]


FID:  8.375099506788018


100%|██████████| 531/531 [00:20<00:00, 26.38it/s]


FID:  7.4673527677825575


100%|██████████| 531/531 [00:20<00:00, 26.14it/s]


FID:  4.863713436322769


100%|██████████| 531/531 [00:20<00:00, 25.57it/s]


FID:  9.783729273222093


In [32]:
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/50/t1" "resources/fid/baseline_val/t1" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/50/t1ce" "resources/fid/baseline_val/t1ce" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/50/t2" "resources/fid/baseline_val/t2" --batch-size 1 --device cuda:0 --num-workers 0 
%run -m pytorch_fid "resources/inpainting_eval/fft100_epoch50/50/flair" "resources/fid/baseline_val/flair" --batch-size 1 --device cuda:0 --num-workers 0 

100%|██████████| 531/531 [00:20<00:00, 26.08it/s]


FID:  7.1421244882321275


100%|██████████| 531/531 [00:20<00:00, 26.50it/s]


FID:  6.208625779067688


100%|██████████| 531/531 [00:20<00:00, 26.30it/s]


FID:  3.978288742802988


100%|██████████| 531/531 [00:19<00:00, 26.93it/s]


FID:  8.334207609269441
